In [1]:
import numpy as np
import pandas as pd
import seaborn.apionly as sns
import matplotlib.pyplot as plt
sns.set()
color = sns.color_palette()

In [2]:
#setup para el notebook

%matplotlib inline
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams['figure.figsize'] = (16, 12)

In [3]:
import os
for data_dir in ["C:\\Users\Rafael\\Documents\\data\\instacart\\raw",
                 "C:/Users/rcrescenzi/Documents/Personal/data/instacart/raw"]:
    if os.path.exists(data_dir):
        print(data_dir)
        break
orders = pd.read_csv(data_dir + "/orders.csv", index_col="order_id")

C:\Users\Rafael\Documents\data\instacart\raw


In [4]:
valid_users = orders[orders.eval_set == "train"].user_id.sample(frac=0.1)
valid_users

order_id
2255645     37891
2699027     34187
3139125     50839
1853036    130635
3356793    189882
            ...  
2033904    200837
895383      11753
2172657    200466
2774552     32668
1496140     99930
Name: user_id, dtype: int64

In [5]:
orders = orders[orders.user_id.isin(valid_users)]
orders

,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,
600894,8,prior,1,6,0,nan
2979257,8,prior,2,1,8,30.00
2570360,8,prior,3,5,0,30.00
880375,8,train,4,1,14,10.00
2625565,21,prior,1,1,10,nan
...,...,...,...,...,...,...
1562848,206178,prior,16,2,22,3.00
1727634,206178,prior,17,5,11,3.00
3239701,206178,prior,18,3,23,12.00


In [6]:
def make_df(path):
    res = []
    for i, cand in enumerate(pd.read_csv(path, chunksize=100000)):
        temp =  cand.join(orders.user_id, on="order_id")
        temp = temp[temp.user_id.isin(valid_users)]
        res.append(temp.join(orders.drop("user_id", axis=1), on="order_id"))
        if (i % 100) == 0:
            print("haciendo", i+1)
            print(temp)
    return pd.concat(res)

In [7]:
train = make_df(data_dir + "/order_products__train.csv")
train

haciendo 1
       order_id  product_id  add_to_cart_order  reordered    user_id
114         226       28199                  1          0  51,011.00
115         226       24852                  2          0  51,011.00
116         226       29883                  3          1  51,011.00
117         226       28427                  4          1  51,011.00
118         226        7754                  5          1  51,011.00
...         ...         ...                ...        ...        ...
99543    245796       36046                  6          0  53,415.00
99544    245796       16553                  7          0  53,415.00
99936    246710       42342                  1          0 190,416.00
99937    246710       13640                  2          0 190,416.00
99938    246710       13014                  3          1 190,416.00

[9972 rows x 5 columns]


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
114,226,28199,1,0,"51,011.00",train,4,0,12,30.00
115,226,24852,2,0,"51,011.00",train,4,0,12,30.00
116,226,29883,3,1,"51,011.00",train,4,0,12,30.00
117,226,28427,4,1,"51,011.00",train,4,0,12,30.00
118,226,7754,5,1,"51,011.00",train,4,0,12,30.00
...,...,...,...,...,...,...,...,...,...,...
1384580,3420998,16185,24,0,"123,299.00",train,25,6,18,30.00
1384581,3420998,6719,25,0,"123,299.00",train,25,6,18,30.00
1384582,3420998,41950,26,0,"123,299.00",train,25,6,18,30.00
1384583,3420998,8174,27,0,"123,299.00",train,25,6,18,30.00


In [13]:
train.user_id.value_counts().sort_index()

8.00          18
21.00          6
24.00          1
52.00         11
65.00          8
              ..
206,113.00    27
206,118.00    10
206,151.00    11
206,167.00    17
206,178.00    14
Name: user_id, dtype: int64

In [14]:
prior = make_df(data_dir + "/order_products__prior.csv")
prior

haciendo 1
       order_id  product_id  add_to_cart_order  reordered    user_id
62            9       21405                  1          0 139,016.00
63            9       47890                  2          1 139,016.00
64            9       11182                  3          0 139,016.00
65            9        2014                  4          1 139,016.00
66            9       29193                  5          1 139,016.00
...         ...         ...                ...        ...        ...
99267     10484        9092                  9          1 181,970.00
99365     10496       37687                  1          1 117,645.00
99366     10496        6000                  2          1 117,645.00
99367     10496       37131                  3          0 117,645.00
99368     10496       43772                  4          0 117,645.00

[5878 rows x 5 columns]
haciendo 101
          order_id  product_id  add_to_cart_order  reordered    user_id
10000000   1055844        2078                 10  

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
62,9,21405,1,0,"139,016.00",prior,14,0,19,5.00
63,9,47890,2,1,"139,016.00",prior,14,0,19,5.00
64,9,11182,3,0,"139,016.00",prior,14,0,19,5.00
65,9,2014,4,1,"139,016.00",prior,14,0,19,5.00
66,9,29193,5,1,"139,016.00",prior,14,0,19,5.00
...,...,...,...,...,...,...,...,...,...,...
32434186,3421043,13249,6,0,"167,283.00",prior,8,2,14,6.00
32434187,3421043,42736,7,1,"167,283.00",prior,8,2,14,6.00
32434188,3421043,33731,8,0,"167,283.00",prior,8,2,14,6.00
32434189,3421043,46979,9,1,"167,283.00",prior,8,2,14,6.00


In [15]:
train.drop(["add_to_cart_order", "eval_set"], axis=1, inplace=True)
train

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
114,226,28199,0,"51,011.00",4,0,12,30.00
115,226,24852,0,"51,011.00",4,0,12,30.00
116,226,29883,1,"51,011.00",4,0,12,30.00
117,226,28427,1,"51,011.00",4,0,12,30.00
118,226,7754,1,"51,011.00",4,0,12,30.00
...,...,...,...,...,...,...,...,...
1384580,3420998,16185,0,"123,299.00",25,6,18,30.00
1384581,3420998,6719,0,"123,299.00",25,6,18,30.00
1384582,3420998,41950,0,"123,299.00",25,6,18,30.00
1384583,3420998,8174,0,"123,299.00",25,6,18,30.00


In [16]:
prior.drop(["add_to_cart_order", "eval_set"], axis=1, inplace=True)
prior

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
62,9,21405,0,"139,016.00",14,0,19,5.00
63,9,47890,1,"139,016.00",14,0,19,5.00
64,9,11182,0,"139,016.00",14,0,19,5.00
65,9,2014,1,"139,016.00",14,0,19,5.00
66,9,29193,1,"139,016.00",14,0,19,5.00
...,...,...,...,...,...,...,...,...
32434186,3421043,13249,0,"167,283.00",8,2,14,6.00
32434187,3421043,42736,1,"167,283.00",8,2,14,6.00
32434188,3421043,33731,0,"167,283.00",8,2,14,6.00
32434189,3421043,46979,1,"167,283.00",8,2,14,6.00


## Creación del dataset de prueba

Para crear un datase sobre el cual hacer predicciones, vamos a tomar los productos comprados en prior y confeccionar un dataset artificial con ellos como productos candidatos a ser recomprados. 

In [17]:
valid_order_data = orders[orders.eval_set == "train"]
valid_order_data = valid_order_data[valid_order_data.user_id.isin(valid_users)]
valid_order_data = valid_order_data.reset_index().set_index("user_id").drop(["eval_set"], axis=1)
valid_order_data

,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
user_id,,,,,
8,880375,4,1,14,10.00
21,1854765,34,1,12,28.00
24,965160,19,0,16,0.00
52,1647290,28,4,16,3.00
65,1072954,15,6,17,30.00
...,...,...,...,...,...
206113,221790,4,1,15,30.00
206118,3371955,24,5,13,8.00
206151,2883005,20,6,21,6.00


In [18]:
def make_dataset(g):
    res = [{
        "user_id": g.user_id.iloc[0],
        "order_id": valid_order_data.loc[g.user_id].order_id.max(),
        "order_number": valid_order_data.loc[g.user_id].order_number.max(),
        "product_id": pid
    } for pid in g.product_id.unique()]
    return res
    


In [19]:
%%time
prior.loc[:10000].groupby("user_id").apply(make_dataset)

Wall time: 5.12 s


user_id
4,209.00      [{'user_id': 4209.0, 'order_id': 1164713, 'ord...
4,645.00      [{'user_id': 4645.0, 'order_id': 2615153, 'ord...
14,386.00     [{'user_id': 14386.0, 'order_id': 1826703, 'or...
18,194.00     [{'user_id': 18194.0, 'order_id': 2351044, 'or...
19,933.00     [{'user_id': 19933.0, 'order_id': 2219929, 'or...
                                    ...                        
191,401.00    [{'user_id': 191401.0, 'order_id': 3066963, 'o...
197,422.00    [{'user_id': 197422.0, 'order_id': 2221847, 'o...
197,745.00    [{'user_id': 197745.0, 'order_id': 2151845, 'o...
204,184.00    [{'user_id': 204184.0, 'order_id': 2760086, 'o...
204,912.00    [{'user_id': 204912.0, 'order_id': 1868438, 'o...
dtype: object

In [20]:
%%time
a = prior.loc[:10000].groupby("user_id").product_id.apply(lambda g: g.unique()).rename("product_id")

Wall time: 7.99 ms


In [21]:
a

user_id
4,209.00      [5876, 26497, 17872, 4675, 21267, 22825, 41588...
4,645.00                            [28745, 22124, 26604, 5212]
14,386.00       [41007, 8324, 15655, 19006, 45681, 16874, 9065]
18,194.00     [20392, 27845, 162, 2452, 8575, 41890, 39475, ...
19,933.00                                        [18615, 19508]
                                    ...                        
191,401.00                         [46931, 26152, 48697, 30486]
197,422.00           [24363, 29625, 22935, 15995, 38890, 28432]
197,745.00    [4210, 4377, 39461, 13627, 13110, 12397, 4840,...
204,184.00                                       [14645, 48226]
204,912.00                          [27845, 24852, 47485, 6532]
Name: product_id, dtype: object

In [22]:
%%time
a = prior.loc[:10000].groupby("user_id").product_id.apply(lambda g: g.unique()).rename("product_id")
a = a.reset_index().groupby("user_id").product_id.apply(lambda x: pd.Series(x.iloc[0]))
a.index.names = ["user_id", "tirar"]
a = a.reset_index().drop("tirar", axis=1).join(valid_order_data, on="user_id")

Wall time: 47 ms


In [23]:
a

,user_id,product_id,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,"4,209.00",5876,1164713,22,2,12,30.00
1,"4,209.00",26497,1164713,22,2,12,30.00
2,"4,209.00",17872,1164713,22,2,12,30.00
3,"4,209.00",4675,1164713,22,2,12,30.00
4,"4,209.00",21267,1164713,22,2,12,30.00
...,...,...,...,...,...,...,...
484,"204,184.00",48226,2760086,9,5,19,5.00
485,"204,912.00",27845,1868438,35,3,10,23.00
486,"204,912.00",24852,1868438,35,3,10,23.00
487,"204,912.00",47485,1868438,35,3,10,23.00


In [24]:
%%time
valid_set = prior.groupby("user_id").product_id.apply(lambda g: g.unique()).rename("product_id")
valid_set = valid_set.reset_index().groupby("user_id").product_id.apply(lambda x: pd.Series(x.iloc[0]))
valid_set.index.names = ["user_id", "tirar"]
valid_set = valid_set.reset_index().drop("tirar", axis=1).join(valid_order_data, on="user_id")

Wall time: 4.24 s


In [25]:
valid_set

,user_id,product_id,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,8.00,21903,880375,4,1,14,10.00
1,8.00,28985,880375,4,1,14,10.00
2,8.00,6473,880375,4,1,14,10.00
3,8.00,39110,880375,4,1,14,10.00
4,8.00,39812,880375,4,1,14,10.00
...,...,...,...,...,...,...,...
844354,"206,178.00",4367,813551,20,4,12,11.00
844355,"206,178.00",3873,813551,20,4,12,11.00
844356,"206,178.00",21775,813551,20,4,12,11.00
844357,"206,178.00",30175,813551,20,4,12,11.00


In [26]:
valid_set = valid_set.join(train.set_index(["order_id", "product_id"]).reordered, on=["order_id", "product_id"])
valid_set

,user_id,product_id,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,8.00,21903,880375,4,1,14,10.00,1.00
1,8.00,28985,880375,4,1,14,10.00,nan
2,8.00,6473,880375,4,1,14,10.00,nan
3,8.00,39110,880375,4,1,14,10.00,nan
4,8.00,39812,880375,4,1,14,10.00,nan
...,...,...,...,...,...,...,...,...
844354,"206,178.00",4367,813551,20,4,12,11.00,nan
844355,"206,178.00",3873,813551,20,4,12,11.00,nan
844356,"206,178.00",21775,813551,20,4,12,11.00,nan
844357,"206,178.00",30175,813551,20,4,12,11.00,nan


In [ ]:
valid_set.reordered.fillna(-1).value_counts()

In [28]:
uid = 8
pd.options.display.max_rows = 20
pd.concat([prior[prior.user_id == uid].product_id.value_counts().rename("prior"),
           train[train.user_id == uid].product_id.value_counts().rename("train"),
           valid_set[valid_set.user_id == uid].product_id.value_counts().rename("valid_set")], axis=1)

,prior,train,valid_set
651,1.00,nan,1.00
1529,1.00,nan,1.00
2078,3.00,nan,1.00
2846,nan,1.00,nan
4799,1.00,nan,1.00
4853,nan,1.00,nan
5539,nan,1.00,nan
6141,1.00,nan,1.00
6473,1.00,nan,1.00
7058,nan,1.00,nan


In [30]:
train[train.user_id == uid].sort_values("product_id")

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
361851,880375,2846,0,8.00,4,1,14,10.00
361839,880375,4853,0,8.00,4,1,14,10.00
361835,880375,5539,0,8.00,4,1,14,10.00
361841,880375,7058,0,8.00,4,1,14,10.00
361836,880375,10960,0,8.00,4,1,14,10.00
361834,880375,15937,1,8.00,4,1,14,10.00
361847,880375,21903,1,8.00,4,1,14,10.00
361838,880375,22247,0,8.00,4,1,14,10.00
361837,880375,23165,1,8.00,4,1,14,10.00
361848,880375,25659,0,8.00,4,1,14,10.00


In [31]:
valid_set.reordered.fillna(0, inplace=True)
valid_set

,user_id,product_id,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,8.00,21903,880375,4,1,14,10.00,1.00
1,8.00,28985,880375,4,1,14,10.00,0.00
2,8.00,6473,880375,4,1,14,10.00,0.00
3,8.00,39110,880375,4,1,14,10.00,0.00
4,8.00,39812,880375,4,1,14,10.00,0.00
5,8.00,20920,880375,4,1,14,10.00,0.00
6,8.00,32030,880375,4,1,14,10.00,0.00
7,8.00,24838,880375,4,1,14,10.00,0.00
8,8.00,2078,880375,4,1,14,10.00,0.00
9,8.00,651,880375,4,1,14,10.00,0.00


In [32]:
valid_set.reordered.value_counts(1)

0.00   0.90
1.00   0.10
Name: reordered, dtype: float64

In [33]:
prior

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
62,9,21405,0,"139,016.00",14,0,19,5.00
63,9,47890,1,"139,016.00",14,0,19,5.00
64,9,11182,0,"139,016.00",14,0,19,5.00
65,9,2014,1,"139,016.00",14,0,19,5.00
66,9,29193,1,"139,016.00",14,0,19,5.00
67,9,34203,1,"139,016.00",14,0,19,5.00
68,9,14992,1,"139,016.00",14,0,19,5.00
69,9,31506,1,"139,016.00",14,0,19,5.00
70,9,23288,0,"139,016.00",14,0,19,5.00
71,9,44533,1,"139,016.00",14,0,19,5.00


In [34]:
from lightgbm import LGBMClassifier

In [103]:
depths = list(range(6, 20, 2))
depths

[6, 8, 10, 12, 14, 16, 18]

In [68]:
def get_real(g): 
    x = g[g.reordered==1]
    x = set(x.product_id.values)
    if len(x) == 0:
        x = set([0])
    return x

real = train.groupby("user_id").apply(get_real)

def get_true_pos(g):
    if not type(g["pred"]) == set:
        g["pred"] = set([0])
    x = g["real"].intersection(g["pred"])
    return x

def get_prec(true_pos, pred):
        if not type(pred) == set:
            pred = [0]
        return len(true_pos) / len(pred)

In [104]:
ths = [i / 10 for i in range(1, 11)]
res = pd.DataFrame([], index=[str(d) for d in depths], columns=ths)

for depth in depths:
    learner = LGBMClassifier(n_estimators=10000, max_depth=depth)
    learner.fit(prior.drop("reordered", axis=1), prior.reordered,
                early_stopping_rounds=10, eval_metric="binary_logloss",
                eval_set=[(prior.drop("reordered", axis=1), prior.reordered),
                          (valid_set.drop("reordered", axis=1), valid_set.reordered)])
    
    for th in ths:
        print(th)
        preds = valid_set[["user_id", "product_id"]].copy()
        preds["preds"] = learner.predict_proba(valid_set.drop("reordered", axis=1))[:, -1]
        preds = preds[preds.preds > th].groupby("user_id").product_id.apply(set)
        preds = pd.concat([real.rename("real"), preds.rename("pred")], axis=1)
        preds["true_pos"] = preds.apply(get_true_pos, axis=1)
        preds["prec"] = preds.apply(lambda g: get_prec(g.true_pos, g.pred), axis=1)
        preds["recall"] = preds.apply(lambda g: len(g.true_pos) / len(g["real"]), axis=1)
        preds["f1"] = 2 * (preds.prec * preds.recall) / (preds.prec + preds.recall)
        preds["f1"] = preds["f1"].fillna(0)
        res.loc[str(depth), th] = preds["f1"].mean()
    print(res)

[1]	valid_0's binary_logloss: 0.671988	valid_1's binary_logloss: 0.727137
Train until valid scores didn't improve in 10 rounds.
[2]	valid_0's binary_logloss: 0.654753	valid_1's binary_logloss: 0.759587
[3]	valid_0's binary_logloss: 0.640563	valid_1's binary_logloss: 0.790114
[4]	valid_0's binary_logloss: 0.628819	valid_1's binary_logloss: 0.819315
[5]	valid_0's binary_logloss: 0.619025	valid_1's binary_logloss: 0.846739
[6]	valid_0's binary_logloss: 0.610821	valid_1's binary_logloss: 0.872776
[7]	valid_0's binary_logloss: 0.603926	valid_1's binary_logloss: 0.8975
[8]	valid_0's binary_logloss: 0.598131	valid_1's binary_logloss: 0.920892
[9]	valid_0's binary_logloss: 0.59322	valid_1's binary_logloss: 0.942493
[10]	valid_0's binary_logloss: 0.589062	valid_1's binary_logloss: 0.962552
[11]	valid_0's binary_logloss: 0.585527	valid_1's binary_logloss: 0.981292
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.671988	valid_1's binary_logloss: 0.727137
0.1
0.2
0.3
0.4
0.5
0.6


0.4
0.5
0.6
0.7
0.8
0.9
1.0
   0.10 0.20 0.30 0.40 0.50 0.60 0.70 0.80 0.90 1.00
6  0.21 0.21 0.21 0.21 0.19 0.07 0.07 0.07 0.07 0.07
8  0.21 0.21 0.21 0.21 0.19 0.07 0.07 0.07 0.07 0.07
10 0.21 0.21 0.21 0.21 0.19 0.07 0.07 0.07 0.07 0.07
12 0.21 0.21 0.21 0.21 0.19 0.07 0.07 0.07 0.07 0.07
14 0.21 0.21 0.21 0.21 0.19 0.07 0.07 0.07 0.07 0.07
16 0.21 0.21 0.21 0.21 0.19 0.07 0.07 0.07 0.07 0.07
18  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
[1]	valid_0's binary_logloss: 0.671983	valid_1's binary_logloss: 0.727132
Train until valid scores didn't improve in 10 rounds.
[2]	valid_0's binary_logloss: 0.654745	valid_1's binary_logloss: 0.75966
[3]	valid_0's binary_logloss: 0.640555	valid_1's binary_logloss: 0.790492
[4]	valid_0's binary_logloss: 0.628799	valid_1's binary_logloss: 0.819547
[5]	valid_0's binary_logloss: 0.619002	valid_1's binary_logloss: 0.846918
[6]	valid_0's binary_logloss: 0.610797	valid_1's binary_logloss: 0.872788
[7]	valid_0's binary_logloss: 0.603893	valid_1's bi